In [ ]:
from pocket_cube.cube import Cube
from pocket_cube.cube import Move
from tests import test_list, test, is_solved
from heuristics import hamming, inverse_hamming, blocked_hamming, blocked_inverse_hamming, manhattan, inverse_manhattan
from utils import get_neighbors, get_path, met_in_the_middle, FrontierItem, DiscoveredDict

from heapq import heappush, heappop
from typing import Callable

%matplotlib notebook


In [ ]:
# A*
def a_star(cube: Cube) -> (list[Move], int):
    # initialize with cube
    frontier: list[FrontierItem] = []
    heappush(frontier, (0 + manhattan(cube), cube.hash(), cube.clone()))
    discovered: DiscoveredDict = {cube.hash(): (None, None, 0)}
    # search
    while frontier:
        currentCube: Cube = heappop(frontier)[2]
        if is_solved(currentCube):
            break
        for (neighbor, move) in get_neighbors(currentCube):
            score: int = discovered[currentCube.hash()][2] + 1
            if neighbor.hash() not in discovered or score < discovered[neighbor.hash()][2]:
                discovered[neighbor.hash()] = (currentCube.hash(), move, score)
                node: FrontierItem = (score + manhattan(neighbor), neighbor.hash(), neighbor.clone())
                heappush(frontier, node)
    # get path
    return (get_path(currentCube.hash(), discovered), len(discovered))

In [ ]:
# Test A*
test(a_star, test_list)

In [ ]:
# Bidirectional BFS
from collections import deque

def bidirectional_bfs(cube: Cube) -> (list[Move], int):
    frontiers: list[deque[Cube]] = [deque(), deque()]
    frontiers[0].append(cube)

    solved_cube = cube.clone()
    solved_cube.state = solved_cube.goal_state
    frontiers[1].append(solved_cube)
    discovereds: list[DiscoveredDict] = [{cube.hash(): (None, None, 0)}, {solved_cube.hash(): (None, None, 0)}]

    while frontiers[0] and frontiers[1]:
        met_cube_key: str = met_in_the_middle(discovereds[0], discovereds[1])
        if met_cube_key is not None:
            break
        currentCubes: tuple[Cube] = (frontiers[0].popleft(), frontiers[1].popleft())
        for i in range(2):
            for (neighbor, move) in get_neighbors(currentCubes[i]):
                score: int = discovereds[i][currentCubes[i].hash()][2] + 1
                if neighbor.hash() not in discovereds[i] or score < discovereds[i][neighbor.hash()][2]:
                    discovereds[i][neighbor.hash()] = (currentCubes[i].hash(), move, score)
                    frontiers[i].append(neighbor)
    path1: list[Move] = get_path(met_cube_key, discovereds[0])
    path2: list[Move] = get_path(met_cube_key, discovereds[1])
    path2.reverse()
    path2 = list(map(Move.opposite, path2))
    return (path1 + path2, len(discovereds[0]) + len(discovereds[1]))

In [ ]:
# Test Bidirectional BFS
test(bidirectional_bfs, test_list)

In [ ]:
# # MTCS with UCB
N = "N"
Q = "Q"
PARENT = "PARENT"
MOVE = "MOVE"
CHILDREN = "CHILDREN"
Node = dict[int, int, Cube, dict[Move, Cube]]
def init_node(parent = None) -> Node:
    return {N: 0, Q: 0, PARENT: parent, CHILDREN: {}}

In [ ]:
from math import sqrt, log

def select_move(node: Node, c) -> Move:
    N_node = node[N]
    max_move: Move = None
    max_expr: float = float('-inf')
    for move in node[CHILDREN]:
        child = node[CHILDREN][move]
        expr = child[Q] / child[N] + c * sqrt(log(N_node) / child[N])
        if expr > max_expr:
            max_expr = expr
            max_move = move
    return max_move

In [ ]:
from random import choice

def mcts(cube0: Cube, budget: int, tree: Node, cp: float, heuristic: Callable[[Cube], int]) -> Node:
    states_visited: int = 0
    if not tree:
        tree = init_node()
    for _ in range(budget):
        cube = cube0
        node = tree
        # go down the tree until a final state or an unexplored move is found
        while not is_solved(cube) and not any([move not in node[CHILDREN] for move in Move]):
            move: Move = select_move(node, cp)
            cube = cube.move(move)
            node = node[CHILDREN][move]
        # if node is not final and not every move has been explored, create a new node
        if not is_solved(cube):
            new_node: Node = init_node(node)
            move: Move = choice([move for move in Move if move not in node[CHILDREN]])
            node[CHILDREN][move] = new_node
            cube = cube.move(move)
            node = new_node
            states_visited += 1
        # simulate a random game
        max_moves: int = 14
        max_h: int = 0
        while not is_solved(cube) and max_moves > 0:
            new_node: Node = init_node(node)
            move: Move = choice([move for move in Move])
            node[CHILDREN][move] = new_node
            cube = cube.move(move)
            node = new_node
            max_h = max(max_h, heuristic(cube))
            max_moves -= 1
            states_visited += 1
        while node:
            node[N] += 1
            node[Q] += max_h
            node = node[PARENT]
    return (tree, states_visited)

def play_mtcs(cube: Cube, budget: int, cp: float, heuristic: Callable[[Cube], int]) -> (list[Move], int):
    (tree, states) = mcts(cube, budget, None, cp, heuristic)
    node: Node = tree
    path: list[Move] = []
    while node and node[CHILDREN]:
        move: Move = select_move(node, 0)
        node = node[CHILDREN][move]
        path.append(move)
    return (path, states)

In [ ]:
# Test MTCS
for heuristic in [inverse_manhattan, inverse_hamming]:
    for c in [0.1, 0.5]:
        for budget in [1000, 5000, 10000, 20000]:
            print(f"Heuristic: {heuristic.__name__} Budget: {budget}, c: {c}")
            test_results: list[list[tuple[bool, float, int, int]]] = []
            for _ in range(0, 20):
                test_results.append(test(lambda cube: play_mtcs(cube, budget, c, heuristic), test_list, False))
            # compute average test result
            for i in range(len(test_list)):
                # average for test i
                test_result: tuple[float, int, int] = (0, 0, 0)
                for result in test_results:
                    test_result = (result[i][1], result[i][2], result[i][3])
                print(f"Test {i} average: Time: {test_result[0] / len(test_results)} seconds. States expanded: {test_result[1] / len(test_results)}. Path length: {test_result[2] / len(test_results)}")